In [1]:
!pip install --upgrade pip
!pip install lightgbm
!pip install pandas
!pip install scikit-learn
!pip install seaborn
!pip install matplotlib
!pip install hyperopt
!pip install \
    'dask>2023.3.2' \
    'distributed>2023.3.2' \
    'dask[dataframe]'

  Using cached pip-25.0.1-py3-none-any.whl.metadata (3.7 kB)
Using cached pip-25.0.1-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.3.1
    Uninstalling pip-24.3.1:
      Successfully uninstalled pip-24.3.1
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
  Using cached dask_expr-1.1.21-py3-none-any.whl.metadata (2.6 kB)
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
  Using cached dask_expr-1.1.20-py3-none-any.whl.metadata (2.6 kB)
  Using cached dask_expr-1.1.19-py3-none-any.whl.metadata (2.6 kB)
  Using cached dask_expr-1.1.18-py3-none-any.whl.metadata (2.6 kB)
  Using cached dask_expr-1.1.16-py3-none-any.whl.metadata (2.5 kB)
  Using cached dask_expr-1.1.15-py3-none-any.whl.metadata (2.5 kB)
  Using cached dask_expr-1.1.14-py3-none-any.whl.metadata (2.5 kB)
  Using cach

In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import  LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from hyperopt import hp
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials
from sklearn.model_selection import cross_val_score

import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor
from datetime import datetime, date, timedelta
import os
import numpy as np
from scipy.stats import lognorm, loguniform, norm
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import cross_validate
import boto3
from io import StringIO
pd.set_option('display.max_columns',100)

In [2]:
curr_dir = os.path.abspath(os.getcwd())
data_dir = os.path.join(curr_dir,'Data')
models_dir = os.path.join(curr_dir,'Models')

os.makedirs(data_dir, exist_ok=True)
os.makedirs(models_dir, exist_ok=True)

In [3]:
df_train_360 = pd.read_csv(os.path.join(data_dir,'KPIs_data_modelling_train_360_days.csv'))
df_test_360 = pd.read_csv(os.path.join(data_dir,'KPIs_data_modelling_test_360_days.csv'))

In [4]:
df_test_360.shape

(6513, 84)

In [5]:
df_train_360.shape

(36904, 84)

In [6]:
df_train_360

,accounts_group,count_units,upfront_price_usd,avg_planned_repayment_days,frr_30,frr_60,frr_90,frr_180,frr_270,frr_360,...,frr_3_years,actual_fr,total_follow_on_revenue_usd,reg_month,country,product_group,area,primary_product,age_in_months,cohort_age
0,2020-06_Kenya_Lanterns_Mwingi_Sun King Pro Eas...,3,11.277135,280,0.071632,0.146520,0.210419,0.347823,0.447212,0.522914,...,0.950346,109.717137,115.449675,2020-06,Kenya,Lanterns,Mwingi,Sun King Pro EasyBuy,55,720
1,2021-04_Kenya_SHS with TV_Malava_Sun King Home...,38,2093.033536,586,0.035413,0.068193,0.097409,0.184375,0.263272,0.344209,...,0.695535,15979.805645,22974.840440,2021-04,Kenya,SHS with TV,Malava,Sun King Home 400 Easybuy GSM,45,720
2,2020-09_Kenya_SHS Entry-Level Upgrade_Nakuru_S...,1,9.222137,371,0.060989,0.145055,0.212088,0.412088,0.629670,0.846429,...,1.000000,167.842902,167.842902,2020-09,Kenya,SHS Entry-Level Upgrade,Nakuru,Sun King Home 120 EasyBuy,52,720
3,2019-10_Tanzania_SHS Entry-Level_Morogoro_Sun ...,35,609.177695,370,0.062964,0.127276,0.186104,0.350813,0.496272,0.645076,...,0.883362,5874.858124,6650.566952,2019-10,Tanzania,SHS Entry-Level,Morogoro,Sun King Home 120 EasyBuy,63,720
4,2018-01_Uganda_SHS Entry-Level_Kabale_Sun King...,12,130.560000,259,0.080729,0.170428,0.248057,0.402158,0.519015,0.601563,...,0.669240,880.749838,1316.044824,2018-01,Uganda,SHS Entry-Level,Kabale,Sun King Home 60 EasyBuy,84,720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36899,2018-12_Nigeria_SHS Entry-Level_Abuja_Sun King...,19,313.238427,280,0.070079,0.116541,0.152207,0.222865,0.282292,0.322634,...,0.471564,1344.178965,2850.469851,2018-12,Nigeria,SHS Entry-Level,Abuja,Sun King Home 60 EasyBuy,73,720
36900,2021-06_Kenya_SHS with TV Upgrade_Kisumu_Sun K...,50,565.813491,587,0.035713,0.070493,0.101834,0.188897,0.277039,0.358506,...,0.725515,20650.479855,28463.201802,2021-06,Kenya,SHS with TV Upgrade,Kisumu,Sun King Home 400 Easybuy GSM,43,720
36901,2021-02_Kenya_SHS Entry-Level Upgrade_Kisumu_S...,77,351.099056,374,0.049717,0.101549,0.152868,0.286804,0.416184,0.536058,...,0.849272,7580.746213,8926.169958,2021-02,Kenya,SHS Entry-Level Upgrade,Kisumu,Sun King Home 60 EasyBuy,47,720
36902,2018-11_Kenya_SHS Entry-Level_Ukunda_Sun King ...,64,1118.136338,280,0.084689,0.157213,0.224453,0.409220,0.568348,0.669886,...,0.864943,6646.781111,7684.646400,2018-11,Kenya,SHS Entry-Level,Ukunda,Sun King Home 60 EasyBuy,74,540


In [7]:
def feature_engineering(df_train, df_test):
    
    # Creating column for Unlock price
    df_train['unlock_price_usd'] = df_train['upfront_price_usd'] + df_train['total_follow_on_revenue_usd']
    df_test['unlock_price_usd'] = df_test['upfront_price_usd'] + df_test['total_follow_on_revenue_usd']
    
    # Calculating averge unlock and upfront price

    df_train['avg_upfront_price_usd'] = np.round((df_train['upfront_price_usd']/df_train['count_units']),0)
    df_train['avg_unlock_price_usd'] = np.round((df_train['unlock_price_usd']/df_train['count_units']),0)

    df_test['avg_upfront_price_usd'] = np.round((df_test['upfront_price_usd']/df_test['count_units']),0)
    df_test['avg_unlock_price_usd'] = np.round((df_test['unlock_price_usd']/df_test['count_units']),0)
    
    # Removing unnecessary columns
    #cols_to_remove = [col for col in df_train.columns if ('repayment_speed_' in col)]
    cols_to_remove = []
    cols_to_remove.append('upfront_price_usd')
    cols_to_remove.append('unlock_price_usd')

    print('columns to remove: ',cols_to_remove)
    
    df_train.drop(cols_to_remove, axis=1, inplace=True)
    df_test.drop(cols_to_remove, axis=1, inplace=True)
    
    print('Sample from train data------------')
    print(df_train.sample(1))
    
    print('Sample from test data------------')
    print(df_test.sample(1))

    df_train = df_train.reset_index().drop(columns = 'index')
    df_test = df_test.reset_index().drop(columns = 'index')
    
    return df_train, df_test

In [8]:
df_train, df_test = feature_engineering(df_train_360, df_test_360)

columns to remove:  ['upfront_price_usd', 'unlock_price_usd']
Sample from train data------------
                                          accounts_group  count_units  \
15017  2019-11_Myanmar (Burma)_SHS Entry-Level_Bago_S...            2   

       avg_planned_repayment_days    frr_30    frr_60    frr_90   frr_180  \
15017                         560  0.054945  0.556319  0.583791  0.666209   

        frr_270   frr_360   frr_450  ...  total_follow_on_revenue_usd  \
15017  0.748626  0.831044  0.913462  ...                   287.915838   

       reg_month          country    product_group  area  \
15017    2019-11  Myanmar (Burma)  SHS Entry-Level  Bago   

                 primary_product  age_in_months  cohort_age  \
15017  Sun King Home 120 EasyBuy             62         720   

       avg_upfront_price_usd  avg_unlock_price_usd  
15017                   13.0                 157.0  

[1 rows x 85 columns]
Sample from test data------------
                                         ac

In [9]:
x_train = df_train.drop(['accounts_group','count_units','frr_3_years','total_follow_on_revenue_usd',
                       'reg_month','country','product_group','area','primary_product','age_in_months',
                      'actual_fr', 'cohort_age'], axis=1)
y_train = df_train[:]['frr_3_years']

print('shape of the input data: ',x_train.shape)
print('shape of the output data: ',y_train.shape)

x_test = df_test.drop(['accounts_group','count_units','frr_3_years','total_follow_on_revenue_usd',
                       'reg_month','country','product_group','area','primary_product','age_in_months',
                      'actual_fr', 'cohort_age'], axis=1)
y_test = df_test[:]['frr_3_years']

print('shape of the input test data: ',x_test.shape)
print('shape of the output test data: ',y_test.shape)

shape of the input data:  (36904, 73)
shape of the output data:  (36904,)
shape of the input test data:  (6513, 73)
shape of the output test data:  (6513,)


In [10]:
x_train.columns

Index(['avg_planned_repayment_days', 'frr_30', 'frr_60', 'frr_90', 'frr_180',
       'frr_270', 'frr_360', 'frr_450', 'frr_540', 'frr_630', 'frr_720',
       'repayment_speed_30', 'repayment_speed_60', 'repayment_speed_90',
       'repayment_speed_180', 'repayment_speed_270', 'repayment_speed_360',
       'repayment_speed_450', 'repayment_speed_540', 'repayment_speed_630',
       'repayment_speed_720', 'avg_cum_days_disabled_30',
       'avg_cum_days_disabled_60', 'avg_cum_days_disabled_90',
       'avg_cum_days_disabled_180', 'avg_cum_days_disabled_270',
       'avg_cum_days_disabled_360', 'avg_cum_days_disabled_450',
       'avg_cum_days_disabled_540', 'avg_cum_days_disabled_630',
       'avg_cum_days_disabled_720', 'at_risk_rate_30', 'at_risk_rate_60',
       'at_risk_rate_90', 'at_risk_rate_180', 'at_risk_rate_270',
       'at_risk_rate_360', 'at_risk_rate_450', 'at_risk_rate_540',
       'at_risk_rate_630', 'at_risk_rate_720', 'disabled_gt_two_week_rate_30',
       'disabled_gt_tw

In [11]:
n_estimators = [50,100,150,200,250,350,450,500,1000,1500]

space = {
    'num_leaves': hp.loguniform("num_leaves", np.log(5), np.log(200)),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(1.5)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 250, 200000, 50),
    'min_child_weight': hp.loguniform('min_child_weight', np.log(0.00001), np.log(1)),
    'min_child_samples': hp.quniform('min_child_samples', 0, 200, 5),
    'n_estimators': hp.choice("n_estimators", n_estimators),
    'max_depth': hp.loguniform('max_depth', np.log(3), np.log(50)),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.1, 1),
    'reg_alpha': hp.uniform('reg_alpha', 0.001, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0.1, 1),
    'subsample': hp.uniform('subsample', 0.1, 1)
}

def hyperparameter_tuning(space):
    
    model = LGBMRegressor(
        n_estimators=int(space['n_estimators']),
        num_leaves=int(space['num_leaves']),
        learning_rate=space['learning_rate'],
        max_depth=int(space['max_depth']),
        subsample_for_bin=int(space['subsample_for_bin']),
        min_child_weight=space['min_child_weight'],
        min_child_samples=int(space['min_child_samples']),
        colsample_bytree=space['colsample_bytree'],
        reg_alpha=space['reg_alpha'],
        reg_lambda=space['reg_lambda'],
        subsample=space['subsample'],
        objective="quantile",
        alpha=0.5,
        n_jobs=-1,
        verbose=-1,
        random_state=1234
    )

    model.set_params(eval_set=[(x_test, y_test)], eval_metric='quantile')

    score_cv = cross_val_score(model, x_train, y_train, cv=5, scoring='neg_root_mean_squared_error').mean()
    
    return {'loss': -score_cv, 'status': STATUS_OK, 'model': model}

trials = Trials()
best_pos_rmse = fmin(fn=hyperparameter_tuning,
                     space=space,
                     algo=tpe.suggest,
                     max_evals=100,
                     trials=trials)

print(best_pos_rmse)

100%|██████████| 100/100 [22:48<00:00, 13.68s/trial, best loss: 0.04799544865114212]
{'colsample_bytree': 0.48612837789165786, 'learning_rate': 0.03363666646274133, 'max_depth': 12.799426554738039, 'min_child_samples': 60.0, 'min_child_weight': 0.00013900522166696805, 'n_estimators': 8, 'num_leaves': 185.84574102396672, 'reg_alpha': 0.9211614583138252, 'reg_lambda': 0.8749195072732266, 'subsample': 0.21685960679782915, 'subsample_for_bin': 154800.0}


In [12]:
best_pos_rmse

{'colsample_bytree': 0.48612837789165786,
 'learning_rate': 0.03363666646274133,
 'max_depth': 12.799426554738039,
 'min_child_samples': 60.0,
 'min_child_weight': 0.00013900522166696805,
 'n_estimators': 8,
 'num_leaves': 185.84574102396672,
 'reg_alpha': 0.9211614583138252,
 'reg_lambda': 0.8749195072732266,
 'subsample': 0.21685960679782915,
 'subsample_for_bin': 154800.0}

In [13]:
#getting the best params

int_params = ['max_depth','min_child_samples','num_leaves','subsample_for_bin']
best_pos_rmse_params = {k: int(v) if k in int_params else v for k,v in best_pos_rmse.items()}
best_pos_rmse_params = {k: int(n_estimators[v]) if k == 'n_estimators' else v for k,v in best_pos_rmse_params.items()}
print(best_pos_rmse_params)

{'colsample_bytree': 0.48612837789165786, 'learning_rate': 0.03363666646274133, 'max_depth': 12, 'min_child_samples': 60, 'min_child_weight': 0.00013900522166696805, 'n_estimators': 1000, 'num_leaves': 185, 'reg_alpha': 0.9211614583138252, 'reg_lambda': 0.8749195072732266, 'subsample': 0.21685960679782915, 'subsample_for_bin': 154800}


In [14]:
model_reg_lgbm_qntl_tuned_pos_rmse = LGBMRegressor(alpha=0.5, boosting_type='gbdt', class_weight=None, 
                                                   device_type='cpu', importance_type='split',n_jobs=-1, 
                                                   num_threads=-1, objective='quantile',random_state=1234,
                                                   **best_pos_rmse_params)

model_reg_lgbm_qntl_tuned_pos_rmse.fit(x_train,y_train, eval_set=(x_test,y_test), eval_metric='quantile')

print(model_reg_lgbm_qntl_tuned_pos_rmse.best_iteration_)
print(model_reg_lgbm_qntl_tuned_pos_rmse.best_score_)
print(model_reg_lgbm_qntl_tuned_pos_rmse.get_params)
print(model_reg_lgbm_qntl_tuned_pos_rmse)
#print(math.sqrt(mean_squared_error(y_val,model_reg_lgbm_qntl_tuned_pos_rmse.predict(x_val))))
#print(math.sqrt(mean_squared_error(y_val_oot,model_reg_lgbm_qntl_tuned_pos_rmse.predict(x_val_oot))))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003898 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17673
[LightGBM] [Info] Number of data points in the train set: 36904, number of used features: 73
[LightGBM] [Info] Start training from score 0.897271
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

In [15]:
def save_model(model_name, model):
    
    model_file_name = "quantile_regression_"+model_name+"_days_"+ str(date.today()) +".txt"
    model.booster_.save_model(os.path.join(models_dir, model_file_name))

In [16]:
save_model('360_'+str(0.5), model_reg_lgbm_qntl_tuned_pos_rmse)

In [17]:
import lightgbm